<a href="https://colab.research.google.com/github/IT21208294/Shortest-Path/blob/Tharushi/SLTshortestPath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simanneal

In [2]:
import pandas as pd
import numpy as np
from simanneal import Annealer
from math import radians, sin, cos, sqrt, atan2
import urllib.parse

In [3]:
# Load CSV data
data = pd.read_csv('/content/locations.csv.txt')

In [4]:
# Extract location names, latitudes, and longitudes
location_names = data['Name'].tolist()
latitudes = data['Latitude'].tolist()
longitudes = data['Longitude'].tolist()

In [5]:
# Number of locations
n_locations = len(location_names)

In [6]:
# Define a function to calculate the Haversine distance between two points
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in kilometers
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [7]:
# Calculate distance matrix
distance_matrix = np.zeros((n_locations, n_locations))
for i in range(n_locations):
    for j in range(n_locations):
        if i != j:
            distance_matrix[i, j] = haversine_distance(latitudes[i], longitudes[i], latitudes[j], longitudes[j])

In [8]:
# Function to set the start node
def set_start_node(route, start_node):
    """Set the start node of the route and move it to the beginning."""
    if start_node != 0:
        route.remove(start_node)
        route.insert(0, start_node)
    return route

In [9]:
# Define a function to calculate the total distance for a given route
def calculate_distance(route):
    distance = 0
    for i in range(len(route) - 1):
        distance += distance_matrix[route[i], route[i + 1]]
    distance += distance_matrix[route[-1], route[0]]
    return distance

In [10]:
# Define a custom simulated annealing class for TSP
class TSPAnnealer(Annealer):
    def __init__(self, initial_route):
        self.route = initial_route
        super().__init__(initial_route)

    def move(self):
        # Swap two cities in the route
        i = np.random.randint(1, len(self.route) - 1)
        j = np.random.randint(1, len(self.route) - 1)
        self.route[i], self.route[j] = self.route[j], self.route[i]

    def energy(self):
        # Return the total distance of the current route
        return calculate_distance(self.route)

In [11]:
# Set the initial route
initial_route = list(range(n_locations))

In [12]:
# Specify the start node
start_node = 0

In [13]:
# Update the initial route to start from the specified start node
initial_route = set_start_node(initial_route, start_node)

In [14]:
# Create an instance of the TSPAnnealer with the initial route
annealer = TSPAnnealer(initial_route)

In [15]:
# Run the simulated annealing algorithm
annealer.auto(minutes=5, steps=1000)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


{'tmax': 81.0,
 'tmin': 1.0000000000000001e-21,
 'steps': 4400000,
 'updates': 100}

In [16]:
# Get the optimized route and distance
optimized_route = annealer.route
optimized_distance = calculate_distance(optimized_route)

In [17]:
# Define a function to generate a Google Maps URL for the optimized route
def generate_google_maps_url(optimized_route, location_names, latitudes, longitudes):
    base_url = "https://www.google.com/maps/dir/?api=1"
    travel_mode = "driving"
    waypoints = []

    # Start point
    start_point = f"{latitudes[optimized_route[0]]},{longitudes[optimized_route[0]]}"

    # Collect waypoints
    for idx in optimized_route:
        waypoint = f"{latitudes[idx]},{longitudes[idx]}"
        waypoints.append(waypoint)

    # Join the waypoints using the pipe ('|') delimiter
    waypoints_str = '|'.join(waypoints)

    # Create the URL
    url = f"{base_url}&origin={start_point}&destination={start_point}&waypoints={urllib.parse.quote(waypoints_str)}&travelmode={travel_mode}"

    return url


In [18]:
# Generate the Google Maps URL
google_maps_url = generate_google_maps_url(optimized_route, location_names, latitudes, longitudes)

# Print the optimized route and distance
print("Optimized Route:")
for idx in optimized_route:
    print(f"{idx}: {location_names[idx]}")  # Print node number and location name
print("Total Distance:", optimized_distance)

# Print the Google Maps URL
print("\nGoogle Maps URL:")
print(google_maps_url)


Optimized Route:
0: Colombo Fort
28: Seema Malaka
44: Crescat Boulevard Mall
38: Dehiwala Zoo
13: Arcade Independence Square
10: BMICH
5: Independence Memorial Hall
48: St. Lucia's Cathedral
46: Gothami Temple
47: Colombo Public Library
29: National Museum of Natural History
36: Grand Mosque
20: Slave Island
32: Viharamahadevi Open Air Amphitheatre
1: Galle Face Green
31: One Galle Face Mall
26: Colombo City Walks
37: Asokaramaya Buddhist Temple
8: Mount Lavinia Beach
21: Wolvendaal Church
23: National Art Gallery
15: National Zoological Gardens of Sri Lanka
6: Kelaniya Raja Maha Vihara
27: The Independence Arcade
9: R. Premadasa Stadium
19: Dutch Hospital Shopping Precinct
41: Royal Colombo Golf Club
49: Rukmani Devi Memorial Hall
17: Colombo Lotus Tower
43: Liberty Plaza Shopping Mall
7: Diyatha Uyana
2: Gangaramaya Temple
14: Old Parliament Building
45: Majestic City Mall
25: Beira Lake
24: Sugathadasa Indoor Stadium
22: Colombo City Centre Mall
18: Pettah Market
3: National Museum 